我们了解了urllib的基本用法，但是其中确实有不方便的地方，比如处理网页验证和Cookies时，需要写Opener和Handler来处理。为了更加方便地实现这些操作，就有了更为强大的库requests，有了它，Cookies、登录验证、代理设置等操作都不是事儿。requests的官方文档有[中文版本](http://cn.python-requests.org/zh_CN/latest/)。

要安装 Requests，只要在你的终端中运行这个简单命令即可：
```
pip install requests
```

# 1. 基础用法 

## 1.1 发送请求

Requests 简便的 API 意味着所有 HTTP 请求类型都是显而易见的。下面我们看看requests的请求方式：

In [ ]:
import requests

r = requests.get('http://httpbin.org/get')#get请求
# r = requests.post('http://httpbin.org/post')#post请求
print(r.text)
# r = requests.put('http://httpbin.org/put')
# r = requests.delete('http://httpbin.org/delete')
# r = requests.head('http://httpbin.org/get')
# r = requests.options('http://httpbin.org/get')

这里分别用get()、post()、put()、delete()等方法实现了POST、PUT、DELETE等请求。是不是比urllib简单太多了？其实这只是冰山一角，更多的还在后面。

## 1.2 GET请求

### 1.2.1 传递参数

In [13]:
import requests
import json
params={
    'wd':'人工智能',
    'hello':'world'
}
url = 'http://httpbin.org/get'
r = requests.get(url=url,params=params)
print(r.url)
print(r.text)
# print(json.loads(r.text))
# print(type(json.loads(r.text)))
# print(r.json())
# print(type(r.json()))#响应体

http://httpbin.org/get?wd=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD&hello=world
{
  "args": {
    "hello": "world", 
    "wd": "\u4eba\u5de5\u667a\u80fd"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.19.1"
  }, 
  "origin": "183.15.178.105", 
  "url": "http://httpbin.org/get?wd=\u4eba\u5de5\u667a\u80fd&hello=world"
}



针对'http://httpbin.org/get?name=germey&age=22 '，一般情况下，这种信息数据会用字典来存储。那么，怎样来构造这个链接呢？这同样很简单，利用params这个参数就好了。

## 1.3 响应信息

Requests 会自动解码来自服务器的内容。大多数 unicode 字符集都能被无缝地解码。请求发出后，Requests 会基于 HTTP 头部对响应的编码作出有根据的推测。以 GitHub 时间线为例：

In [5]:
import requests
r = requests.get('https://api.github.com/events')
print(type(r.text))#使用text可以获得，Requests 会使用其推测的文本编码
# print(r.text)
print(r.encoding)

<class 'str'>
utf-8


### 1.3.1 响应属性信息

有很多属性和方法可以用来获取其他信息，比如状态码、响应头、Cookies等。示例如下

In [7]:
import requests

r = requests.get('https://www.douban.com')#get请求
print(type(r.status_code),r.status_code)#状态码
print(type(r.headers),r.headers)#响应头
print(type(r.cookies),r.cookies)#cookies
print(type(r.url),r.url)#url
# print(type(r.history),r.history)#history属性得到请求历史
requests.utils.dict_from_cookiejar(r.cookies)

<class 'int'> 200
<class 'requests.structures.CaseInsensitiveDict'> {'Date': 'Thu, 13 Sep 2018 03:15:55 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=30', 'Vary': 'Accept-Encoding', 'X-Xss-Protection': '1; mode=block', 'X-Douban-Mobileapp': '0', 'Expires': 'Sun, 1 Jan 2006 01:00:00 GMT', 'Pragma': 'no-cache', 'Cache-Control': 'must-revalidate, no-cache, private', 'Set-Cookie': 'll="118282"; path=/; domain=.douban.com; expires=Fri, 13-Sep-2019 03:15:55 GMT, bid=Hf0wmwusFBk; Expires=Fri, 13-Sep-19 03:15:55 GMT; Domain=.douban.com; Path=/', 'X-DOUBAN-NEWBID': 'Hf0wmwusFBk', 'X-DAE-Node': 'brand13', 'X-DAE-App': 'sns', 'Server': 'dae', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=15552000;', 'Content-Encoding': 'gzip'}
<class 'requests.cookies.RequestsCookieJar'> <RequestsCookieJar[<Cookie bid=Hf0wmwusFBk for .douban.com/>, <Cookie ll="118282" for .douban.com/>]>
<class 'str'> h

{'bid': 'Hf0wmwusFBk', 'll': '"118282"'}

状态码常用来判断请求是否成功，而requests还提供了一个内置的状态码查询对象requests.codes，示例如下：

In [ ]:
import requests
 
r = requests.get('http://www.douban.com/help.html')
if r.status_code == requests.codes.okay:
    print('存在')
else:
    print('Request Successfully')
# exit() if r.status_code == requests.codes.not_found else print('Request Successfully')

In [20]:
import requests
 
r = requests.get('http://www.douban.com')
print('sahdjkf') if not r.status_code == 404 else print('404 Not Found')

sahdjkf


下面列出了返回码和相应的查询条件：

```
# 信息性状态码
100: ('continue',),
101: ('switching_protocols',),
102: ('processing',),
103: ('checkpoint',),
122: ('uri_too_long', 'request_uri_too_long'),
 
# 成功状态码
200: ('ok', 'okay', 'all_ok', 'all_okay', 'all_good', '\\o/', '✓'),
201: ('created',),
202: ('accepted',),
203: ('non_authoritative_info', 'non_authoritative_information'),
204: ('no_content',),
205: ('reset_content', 'reset'),
206: ('partial_content', 'partial'),
207: ('multi_status', 'multiple_status', 'multi_stati', 'multiple_stati'),
208: ('already_reported',),
226: ('im_used',),
 
# 重定向状态码
300: ('multiple_choices',),
301: ('moved_permanently', 'moved', '\\o-'),
302: ('found',),
303: ('see_other', 'other'),
304: ('not_modified',),
305: ('use_proxy',),
306: ('switch_proxy',),
307: ('temporary_redirect', 'temporary_moved', 'temporary'),
308: ('permanent_redirect',
      'resume_incomplete', 'resume',), # These 2 to be removed in 3.0
 
# 客户端错误状态码
400: ('bad_request', 'bad'),
401: ('unauthorized',),
402: ('payment_required', 'payment'),
403: ('forbidden',),
404: ('not_found', '-o-'),
405: ('method_not_allowed', 'not_allowed'),
406: ('not_acceptable',),
407: ('proxy_authentication_required', 'proxy_auth', 'proxy_authentication'),
408: ('request_timeout', 'timeout'),
409: ('conflict',),
410: ('gone',),
411: ('length_required',),
412: ('precondition_failed', 'precondition'),
413: ('request_entity_too_large',),
414: ('request_uri_too_large',),
415: ('unsupported_media_type', 'unsupported_media', 'media_type'),
416: ('requested_range_not_satisfiable', 'requested_range', 'range_not_satisfiable'),
417: ('expectation_failed',),
418: ('im_a_teapot', 'teapot', 'i_am_a_teapot'),
421: ('misdirected_request',),
422: ('unprocessable_entity', 'unprocessable'),
423: ('locked',),
424: ('failed_dependency', 'dependency'),
425: ('unordered_collection', 'unordered'),
426: ('upgrade_required', 'upgrade'),
428: ('precondition_required', 'precondition'),
429: ('too_many_requests', 'too_many'),
431: ('header_fields_too_large', 'fields_too_large'),
444: ('no_response', 'none'),
449: ('retry_with', 'retry'),
450: ('blocked_by_windows_parental_controls', 'parental_controls'),
451: ('unavailable_for_legal_reasons', 'legal_reasons'),
499: ('client_closed_request',),
 
# 服务端错误状态码
500: ('internal_server_error', 'server_error', '/o\\', '✗'),
501: ('not_implemented',),
502: ('bad_gateway',),
503: ('service_unavailable', 'unavailable'),
504: ('gateway_timeout',),
505: ('http_version_not_supported', 'http_version'),
506: ('variant_also_negotiates',),
507: ('insufficient_storage',),
509: ('bandwidth_limit_exceeded', 'bandwidth'),
510: ('not_extended',),
511: ('network_authentication_required', 'network_auth', 'network_authentication')
```

### 1.3.2 二进制响应内容

图片、音频、视频这些文件本质上都是由二进制码组成的，由于有特定的保存格式和对应的解析方式，我们才可以看到这些形形色色的多媒体。所以，想要抓取它们，就要拿到它们的二进制码。

In [ ]:
import requests

headers ={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
}

 
r = requests.get("http://mm.chinasareview.com/wp-content/uploads/2017a/08/02/01.jpg",headers=headers)
print(r.status_code)
print(r.text)
print(r.content)
with open('05.jpg','wb') as f:
    f.write(r.content)

可以注意到，前者出现了乱码，后者结果前带有一个b，这代表是bytes类型的数据。由于图片是二进制数据，所以前者在打印时转化为str类型，也就是图片直接转化为字符串，这理所当然会出现乱码。

接着，我们将刚才提取到的图片保存下来：

In [ ]:
import requests
 
r = requests.get("https://github.com/favicon.ico")
print(r.status_code)
print(r.content)
with open('favicon.ico', 'wb') as f:
    f.write(r.content)

### 1.3.3  json响应内容

Requests 中也有一个内置的 JSON 解码器，助你处理 JSON 数据：

In [ ]:
import requests
import json

r = requests.get('https://api.github.com/get')
print(type(r.text))
print(r.text)
print(type(r.json()))
print(r.json())
print(type(json.loads(r.text)))
print(json.loads(r.text))

可以发现，调用json()方法，就可以将返回结果是JSON格式的字符串转化为字典。

## 1.4 添加headers

比如，在上面“知乎”的例子中，如果不传递headers，就不能正常请求：

如果你想为请求添加 HTTP 头部，只要简单地传递一个 dict 给 headers 参数就可以了。

In [26]:
import requests
 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
}
r = requests.get("https://www.zhihu.com/explore", headers=headers)
# print(r.text)
print(r.status_code)

200


In [35]:
import requests

url='http://fm.shiyunjj.com/2019/1588/3ikq.jpg'

headers = {
    'Referer':'http://www.mmjpg.com/mm/1588/3',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}

r=requests.get(url=url,headers=headers,allow_redirects=False)
print(r.status_code)
# print(r.content)
with open('aab.jpg','wb') as f:
    f.write(r.content)

200


## 1.5 POST请求

In [40]:
help(requests.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary (will be form-encoded), bytes, or file-like object to send in the body of the :class:`Request`.
    :param json: (optional) json data to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



通常，你想要发送一些编码为表单形式的数据——非常像一个 HTML 表单。要实现这个，只需简单地传递一个字典给 data 参数。你的数据字典在发出请求时会自动编码为表单形式

In [43]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'
}
data ={
    'wd':'python',
    'aaaa':'bbbbb'
}
r=requests.post(url='http://httpbin.org/post',params={'wd':'python'},headers=headers,data=data)
print(r.text)
print(r.url)

{
  "args": {
    "wd": "python"
  }, 
  "data": "", 
  "files": {}, 
  "form": {
    "aaaa": "bbbbb", 
    "wd": "python"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Content-Length": "20", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
  }, 
  "json": null, 
  "origin": "183.15.178.105", 
  "url": "http://httpbin.org/post?wd=python"
}

http://httpbin.org/post?wd=python


# 2. 高级用法 

我们再来了解下requests的一些高级用法，如文件上传、cookie设置、代理设置等。

## 2.1 上传文件

我们知道requests可以模拟提交一些数据。假如有的网站需要上传文件，我们也可以用它来实现，这非常简单，示例如下：

In [ ]:
import requests
 
files = {'file': open('cookies.txt', 'rb')}
r = requests.post("http://httpbin.org/post", files=files)
print(r.text)

# 2.2 模拟登陆

### 2.2.1 用session模拟登陆

In [64]:
import requests
url = 'https://www.zhihu.com/signup?next=%2F'

headers= {
    "HOST": "www.zhihu.com",
    "Referer": "https://www.zhihu.com/",
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}

data={'username':'13027937678','password':'Zyl17727478517'}
s = requests.session()
s.post(url,data=data,headers=headers)
r=s.get('https://www.zhihu.com',headers=headers,allow_redirects=False)
print(r.content.decode('utf-8'))

Redirecting to <a href="https://www.zhihu.com/signup?next=%2F">https://www.zhihu.com/signup?next=%2F</a>.


#### 2.2.2 把cookie放到headers里面进行模拟登陆

In [63]:
import requests
url = 'https://www.zhihu.com/'

headers={    
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
    'Cookie': '_xsrf=F5xzrmVujpiUFdKkH9kTG1pRlIWX9pR7; d_c0="ABDpbgWLeA6PTtOYbPKKw7h-eLdJ0cqiLFw=|1541387675"; _zap=f89f7cbb-8441-4bb3-b46e-b49d8947bcf8; q_c1=6e330507b25246f0ae679ff1cc7cca33|1544151065000|1541387672000; tst=r; __gads=ID=282b119eac68f728:T=1546934837:S=ALNI_Mb7CI5D8aRSfli913mSnofF0c-1AQ; __utma=155987696.1947136852.1546935188.1546935188.1546935188.1; __utmb=155987696.0.10.1546935188; __utmc=155987696; __utmz=155987696.1546935188.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); tgw_l7_route=8ffa4a0b7ecd9bdb5ad19b8c1037b063; capsion_ticket="2|1:0|10:1546936452|14:capsion_ticket|44:Nzc3ZGM0ZWJjOTU4NGJiZmI5ZWRhZDE4YjE4YjY5NzE=|380f1c35f84f4cacec30cb9ab6430fe44f7b90fc9cb6b52ea7a135373aa023e4"; z_c0="2|1:0|10:1546936461|4:z_c0|92:Mi4xbzJQZ0RRQUFBQUFBRU9sdUJZdDREaVlBQUFCZ0FsVk5qYTRoWFFCS05GZGJUQi1oeUo0QXpJbk9oQ3QwQlVGandB|ed811f32c42a4fe09462b57b3a85b759e912334ac36015c807ed30f95a408ec6"'
}
r=requests.get(url,headers=headers,allow_redirects=False)
print(r.content.decode('utf-8'))

<!doctype html>
<html lang="zh" data-hairline="true" data-theme="light"><head><meta charSet="utf-8"/><title data-react-helmet="true">首页 - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><title>知乎 - 有问题上知乎</title><meta name="description" content="有问题，上知乎。知乎是中文互联网知名知识分享平台，以「知识连接一切」为愿景，致力于构建一个人人都可以便捷接入的知识分享网络，让人们便捷地与世界分享知识、经验和见解，发现更大的世界。"/><link rel="shortcut icon" type="image/x-icon" href="https://static.zhihu.com/static/favicon.ico"/><link rel="search" type="application/opensearchdescription+xml" href="https://static.zhihu.com/static/search.xml" title="知乎"/><link rel="dns-prefetch" href="//static.zhimg.com"/><link rel="dns-prefetch" href="//pic1.zhimg.com"/><link rel="dns-prefetch" href="//pic2.zhimg.co

### 2.2.3 将cookie值拿出来直接在requests中发送

In [72]:
import requests
url = 'https://www.zhihu.com/'

headers={    
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}
cc={}
cookie= '_xsrf=F5xzrmVujpiUFdKkH9kTG1pRlIWX9pR7; d_c0="ABDpbgWLeA6PTtOYbPKKw7h-eLdJ0cqiLFw=|1541387675"; _zap=f89f7cbb-8441-4bb3-b46e-b49d8947bcf8; q_c1=6e330507b25246f0ae679ff1cc7cca33|1544151065000|1541387672000; tst=r; __gads=ID=282b119eac68f728:T=1546934837:S=ALNI_Mb7CI5D8aRSfli913mSnofF0c-1AQ; __utma=155987696.1947136852.1546935188.1546935188.1546935188.1; __utmb=155987696.0.10.1546935188; __utmc=155987696; __utmz=155987696.1546935188.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); tgw_l7_route=8ffa4a0b7ecd9bdb5ad19b8c1037b063; capsion_ticket="2|1:0|10:1546936452|14:capsion_ticket|44:Nzc3ZGM0ZWJjOTU4NGJiZmI5ZWRhZDE4YjE4YjY5NzE=|380f1c35f84f4cacec30cb9ab6430fe44f7b90fc9cb6b52ea7a135373aa023e4"; z_c0="2|1:0|10:1546936461|4:z_c0|92:Mi4xbzJQZ0RRQUFBQUFBRU9sdUJZdDREaVlBQUFCZ0FsVk5qYTRoWFFCS05GZGJUQi1oeUo0QXpJbk9oQ3QwQlVGandB|ed811f32c42a4fe09462b57b3a85b759e912334ac36015c807ed30f95a408ec6"'
for c in cookie.split(';'):
    cc[c.strip().split('=',maxsplit=1)[0]]=c.strip().split('=',maxsplit=1)[1]
r=requests.get(url,headers=headers,allow_redirects=False,cookies=cc)
print(r.content.decode('utf-8'))

<!doctype html>
<html lang="zh" data-hairline="true" data-theme="light"><head><meta charSet="utf-8"/><title data-react-helmet="true">首页 - 知乎</title><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1"/><meta name="renderer" content="webkit"/><meta name="force-rendering" content="webkit"/><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="google-site-verification" content="FTeR0c8arOPKh8c5DYh_9uu98_zJbaWw53J-Sch9MTg"/><title>知乎 - 有问题上知乎</title><meta name="description" content="有问题，上知乎。知乎是中文互联网知名知识分享平台，以「知识连接一切」为愿景，致力于构建一个人人都可以便捷接入的知识分享网络，让人们便捷地与世界分享知识、经验和见解，发现更大的世界。"/><link rel="shortcut icon" type="image/x-icon" href="https://static.zhihu.com/static/favicon.ico"/><link rel="search" type="application/opensearchdescription+xml" href="https://static.zhihu.com/static/search.xml" title="知乎"/><link rel="dns-prefetch" href="//static.zhimg.com"/><link rel="dns-prefetch" href="//pic1.zhimg.com"/><link rel="dns-prefetch" href="//pic2.zhimg.co

前面我们使用urllib处理过Cookies，写法比较复杂，而有了requests，获取和设置Cookies只需一步即可完成。

当然，你也可以通过cookies参数来设置，不过这样就需要构造RequestsCookieJar对象，而且需要分割一下cookies。这相对烦琐，不过效果是相同的

测试后，发现同样可以正常登录天猫。

## 2.3 会话维持

在requests中，如果直接利用get()或post()等方法的确可以做到模拟网页的请求，但是这实际上是相当于不同的会话，也就是说相当于你用了两个浏览器打开了不同的页面。

In [45]:
import requests
 
requests.get('http://httpbin.org/cookies/set/number/123456789')
r = requests.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {}
}



其实解决这个问题的主要方法就是维持同一个会话，也就是相当于打开一个新的浏览器选项卡而不是新开一个浏览器。但是我又不想每次设置cookies，那该怎么办呢？这时候就有了新的利器——Session对象。

In [46]:
import requests
 
s = requests.Session()#会话    
s.get('http://httpbin.org/cookies/set/number/123456789')
r = s.get('http://httpbin.org/cookies')
print(r.text)

{
  "cookies": {
    "number": "123456789"
  }
}



## 2.4 SSL 证书验证

Requests 可以为 HTTPS 请求验证 SSL 证书，就像 web 浏览器一样。SSL 验证默认是开启的，如果证书验证失败，Requests 会抛出 SSLError:

12306的证书没有被官方CA机构信任，会出现证书验证错误的结果。我们现在访问它，都可以看到一个证书问题的页面

In [74]:
import requests
 
r = requests.get('https://www.12306.cn')
print(r.text)

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <title>ä¸­å½éè·¯12306</title>
    <script>
        window.startTime = new Date().getTime(); //"window.onloadå¤å¼å§æ¶é´:",window.startTime
    </script>
    <link rel="shortcut icon" href="images/favicon.ico" type="image/x-icon" />
    <!-- <link href="./css/index.css" rel="stylesheet">
    <link href="./css/global.css" rel="stylesheet">
    <link href="./css/public.css" rel="stylesheet"> -->
    <link href="./fonts/iconfont.css" rel="stylesheet">
    <!-- æ¥æåå¸æ§ä»¶ -->
    <!-- <link href="./css/common/calendarNew.css" rel="stylesheet">
    <link href="./css/common/table.css" rel="stylesheet">
    <link href="./css/common/station.css" rel="stylesheet">
    <link rel="stylesheet" href="./css/main.css"> -->
    <link rel="stylesheet" href="./css/index_y_v30001.css">
</head>
<style>
    .typeahead li {
        width: 265px;
        padding-left:

这里提示一个错误SSLError，表示证书验证错误。所以，如果请求一个HTTPS站点，但是证书验证错误的页面时，就会报这样的错误，那么如何避免这个错误呢？很简单，把verify参数设置为False即可。

In [ ]:
import requests
 
r = requests.get('https://www.12306.cn',verify=False)
print(r.text)
print(r.url)

当然，我们也可以指定一个本地证书用作客户端证书，这可以是单个文件（包含密钥和证书）或一个包含两个文件路径的元组,方法参照官方文档里面的[客户端证书](http://cn.python-requests.org/zh_CN/latest/user/advanced.html#id4)。

## 2.5 代理

对于某些网站，在测试的时候请求几次，能正常获取内容。但是一旦开始大规模爬取，对于大规模且频繁的请求，网站可能会弹出验证码，或者跳转到登录认证页面，更甚者可能会直接封禁客户端的IP，导致一定时间段内无法访问。

使用的代理ip可以从[西刺](http://www.xicidaili.com/)获取。

那么，为了防止这种情况发生，我们需要设置代理来解决这个问题，这就需要用到proxies参数。可以用这样的方式设置：

In [73]:
import requests
 
# proxies={    
#     'http':'http://118.190.95.43:9001',
#     'https':'http://122.96.93.158:49435'  
# }
 
r = requests.get('http://httpbin.org/get')
print(r.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.19.1"
  }, 
  "origin": "183.15.178.105", 
  "url": "http://httpbin.org/get"
}



In [74]:
import requests

proxies = {
    'http':'http://171.41.82.86:9999'
}
r=requests.get('http://httpbin.org/get',proxies=proxies)
print(r.text)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Cache-Control": "max-age=259200", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.19.1"
  }, 
  "origin": "171.41.82.86", 
  "url": "http://httpbin.org/get"
}



其他的设置方法见[官方文档](http://cn.python-requests.org/zh_CN/latest/user/advanced.html#proxies)。

## 2.6 自定义身份验证

Requests 允许你使用自己指定的身份验证机制。详情见[官方文档](http://cn.python-requests.org/zh_CN/latest/user/advanced.html#custom-auth)。

任何传递给请求方法的 auth 参数的可调用对象，在请求发出之前都有机会修改请求

In [37]:
import requests
from requests.auth import HTTPBasicAuth
 
r = requests.get('https://mail.163.com/', auth=HTTPBasicAuth('17727478517@163.com', 'Youlzhang315088'))
print(r.status_code)
print(r.url)

200
https://mail.163.com/


当然，如果参数都传一个HTTPBasicAuth类，就显得有点烦琐了，所以requests提供了一个更简单的写法，可以直接传一个元组，它会默认使用HTTPBasicAuth这个类来认证。

所以上面的代码可以直接简写如下：

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
 
r = requests.get('http://pizzabin.org/admin', auth=('username', 'password'))
print(r.status_code)

## 2.7 Prepared Request

前面介绍urllib时，我们可以将请求表示为数据结构，其中各个参数都可以通过一个Request对象来表示。这在requests里同样可以做到，这个数据结构就叫[Prepared Request](http://cn.python-requests.org/zh_CN/latest/user/advanced.html#prepared-request)。我们用实例看一下：

In [ ]:
from requests import Request, Session
 
url = 'http://httpbin.org/post'
data = {
    'name': 'germey'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36'
}
s = Session()
req = Request('POST', url, data=data, headers=headers)
#print(req)
prepped = s.prepare_request(req)
r = s.send(prepped)
print(r.text)

有了Request这个对象，就可以将请求当作独立的对象来看待，这样在进行队列调度时会非常方便。

## 2.8 超时

在本机网络状况不好或者服务器网络响应太慢甚至无响应时，我们可能会等待特别久的时间才可能收到响应，甚至到最后收不到响应而报错。为了防止服务器不能及时响应，应该设置一个超时时间，即超过了这个时间还没有得到响应，那就报错。这需要用到timeout参数。这个时间的计算是发出请求到服务器返回响应的时间。示例如下：

In [76]:
import requests
 
r = requests.get("https://www.google.com",timeout=0.1)
print(r.status_code)

ConnectTimeout: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x0000000005315EF0>, 'Connection to www.google.com timed out. (connect timeout=0.1)'))

这一 timeout 值将会用作 connect 和 read 二者的 timeout。如果要分别制定，就传入一个元组：

In [1]:
import requests
 
r = requests.get("https://www.taobao.com", timeout =(1,1))
print(r.status_code)

200


如果远端服务器很慢，你可以让 Request 永远等待，传入一个 None 作为 timeout 值，然后就冲咖啡去吧。

In [39]:
import requests
 
r = requests.get('https://www.google.com', timeout=None)
print(r.status_code)

ConnectTimeout: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000000000577A208>, 'Connection to www.google.com timed out. (connect timeout=1)'))

# 3. 异常

关于异常的[官方文档](http://cn.python-requests.org/zh_CN/latest/api.html#id3)

In [94]:
import requests
from requests.exceptions import Timeout,HTTPError,RequestException,ConnectionError,ReadTimeout

try:
    r = requests.get('http://www.youliang.com')
    print(r.text)
except ReadTimeout:
    print('TimeOut')
except HTTPError:
    print('Http Error')
except ConnectionError:
    print('Connection Error')
except RequestException:
    print('Error')

Error


In [96]:
help(UserAgent)

Help on class FakeUserAgent in module fake_useragent.fake:

class FakeUserAgent(builtins.object)
 |  FakeUserAgent(cache=True, use_cache_server=True, path='C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\fake_useragent_0.1.11.json', fallback=None, verify_ssl=True, safe_attrs=())
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, attr)
 |  
 |  __getitem__(self, attr)
 |  
 |  __init__(self, cache=True, use_cache_server=True, path='C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\fake_useragent_0.1.11.json', fallback=None, verify_ssl=True, safe_attrs=())
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  load(self)
 |  
 |  update(self, cache=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [122]:
from fake_useragent import UserAgent
ua = UserAgent()
ua.random

'Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36'